
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement et l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, fourni par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. Les ressources  du projet peuvent être utilisées dans tout autre environnement compatible.![licence : Creative Commons](images/Licence.jpg) 

Auteurs : G. Vidal, C-H. Eyraud, E. le Jan

# Mesurer avec un capteur Press-Hygro-Temp : Adafruit BME280#

Activité réalisée avec un capteur Adafruit BME280, une raspberry Pi3 fonctionnant avec une [image Debian Stretch fournie par l'IFÉ ENS de Lyon](http://mediaserv.climatetmeteo.fr/images/RaspBerry/), disposant des librairies INTEL mraa et upm.

Sur le site de Adafruit https://learn.adafruit.com/adafruit-bme280-humidity-barometric-pressure-temperature-sensor-breakout/pinouts on lit que 4 broches doivent être connectées au raspberryPi lorsqu'on travaille avec le bus I2C.

* Vin - C'est la broche d'alimentation : elle peut-être relié à 3,3V ou 5V (Nous le relierons au 3,3V dans cet exercice). 
* GND - broche à relier à une broche ground du RasberryPi
* SCK - c'est la broche d'horloge du bus I2C à relier à la broche 5 (SCL)
* SDI - c'est la broche de données du bus I2C à relier à la broche 3 (SDA)



![Capteur BME280 branché sur le bus I2C d'un RaspberryPi](images/RaspberryPi_BME280.png)



## Mesurer puis écrire la température simplement en 6 lignes de code ##

Chaque ligne de texte ci-dessous est traduite dans le bloc suivant en une instruction dans un langage interprétable par la machine, à chaque alinéa correspond une instruction, la somme des instructions constitue un programme.

 * (l. 1) Demander l'utilisation des outils nécessaires, il s'agit de deux bibliothèques de programmes de base qui permettent d'interagir et de piloter les capteurs que nous avons choisis. Le nom de ces bibliothèques est mraa et upm
 * (l. 2) Parmi tous les pilotes disponibles nous devons utiliser le pilote pyupm_bmp280 (py pour python, upm nom de la librairie, _ séparateur, bme280 nom du capteur)
 * (l. 3) Déclarer le nom du capteur qu'on va utiliser (bme)
 * (l. 4) Mettre à jour le capteur (on utilise la commande préprogrammée dans le pilote "update")
 * (l. 5) Mesurer la température (on utilise la fonction préprogrammée dans le pilote "getTemperature()" on appelle la température tempBME)
 * (l. 6) Imprimer la température

In [15]:
import mraa, upm
from upm import pyupm_bmp280 as bmp
bme = bmp.BME280(0,0x77)
bme.update()
tempBME = bme.getTemperature()
print ("{0:.2f} °C".format(tempBME))

20.19 °C


## Afficher la température mesurée sur  l'afficheur

Le bloc de code suivant permet d'afficher la température mesurée  par le capteur BME280 sur l'afficheur du cahier d'exercice voisin. L'affichage se fait en gris pendant 10 secondes.
Le premier bloc reprend les déclarations explicitées ci-dessus en ajoutant les instructions nécessaires pour utiliser l'afficheur (voir le cahier dédié à l'afficheur). Pour plus de clarté (ce qui rajoute des lines de code) les adresses des capteurs et effecteurs sont passées par des variables :  lcdAddress, rgbAddress, bmeAddr, bus1.

![Affichage des mesures du Capteur BME280](images/RaspberryPi_BME280Ecran.png)

In [16]:
import mraa, time
from upm import pyupm_bmp280 as bmp
from upm import pyupm_jhd1313m1 as jdm

lcdAddress = 0x3E
rgbAddress = 0x62
bus1 = 0
bmeAddr = 0x77

bme = bmp.BME280(bus1,bmeAddr)
lcd = jdm.Jhd1313m1(bus1,lcdAddress,rgbAddress)

status = lcd.clear()
status = lcd.backlightOff()

Le bloc suivant crée le dessin du carcatère degré car ce caractère n'existe pas par défaut dans l'afficheur. Le dessin est réalisé en codant la valeur (allumé ou éteint) de chacun des pixels.

In [17]:
# création et enregistrement du caractère degré
degree = (
    0b01100,
    0b10010,
    0b01100,
    0b00000,
    0b00000,
    0b00000,
    0b00000,
    0b00000)

lcd.createChar(0,degree)

0

## Afficher Température et pression sur l'afficheur

Le bloc suivant réalise la mesure de température qui est stockée dans la variable tempBME dont la valeur est ensuite affichée sur l'afficheur.
Les mesures sont affichées sur l'afficheur en utilisant le caractère spécial `degré` qui est dessiné  et ajouté.

In [18]:
# Mise à jour du capteur
bme.update()
#Mesure de la Température de la Pression et de l'humidité
tempBME = bme.getTemperature()
pressBME = bme.getPressure() / 100.0
hygroBME = bme.getHumidity()

#positionnement de l'affichage
status = lcd.backlightOff()
status = lcd.clear()
lcd.setColor(255,0,0)
lcd.setCursor(1, 0)
#affichage du texte et de la valeur
status = lcd.backlightOn()
status = lcd.write("{0:.2f}".format(tempBME))
status = lcd.write(chr(0))
status = lcd.write("C {0:.2f} %".format(hygroBME))
lcd.setCursor(2, 0)
status = lcd.write("P : {0:6.2f} hPa".format(pressBME))

#maintien de l'affichage pendant un arc enciel de 76.5 secondes
for i in range(0,255,1) :
    lcd.setColor(255,i,0)
    time.sleep(0.05)
for i in range(0,255,1) :
    lcd.setColor(255-i,255,0)
    time.sleep(0.05)
for i in range(0,255,1) :
    lcd.setColor(0,255,i)
    time.sleep(0.05)
for i in range(0,255,1) :
    lcd.setColor(0,255-i,255)
    time.sleep(0.05)
for i in range(0,255,1) :
    lcd.setColor(i,0,255)
    time.sleep(0.05)
for i in range(0,255,1) :
    lcd.setColor(255,0,255-i)
    time.sleep(0.05)

# effacement et extinction
status = lcd.clear()
status = lcd.backlightOff()

## Ajouter une fonction pour calculer la pression au niveau de la mer ##

La pression atmosphérique en un lieu dépend de son altitude. Pour pouvoir faire de la météorologie ou simplement comparer des mesures faites en différents endroits il faut partager une référence commune et le niveau de la mer est une solution simple et efficace. On convertit donc  notre mesure locale à la valeur qu'elle aurait  si nous étions au niveau de la mer. Il nous faut donc connaître notre altitude et importer des outils mathématiques. pour cela on ajoute la constante localAlt au programme et on importe la bibliothèque d'outils mathématiques de python.
Il est probable que nous ayons à réutiliser ce calcul  dans d'autres programmes et il serait malin de pouvoir réutiliser simplement les lignes de code que nous allons écrire. Plutôt que les noyer dans le programme nous allons _définir une fonction_ qui prendra un _paramètre_ (la pression mesurée m_Pressure) et qui nous _retournera_ la valeur à laquelle cette mesure correspond au niveau de la mer (sl_Pressure pour sea-level pressure). Le paramètre est une variable tout comme la valeur retournée.
La fonction est appelée ici convertSeaLevel() elle prend comme paramètre la variable mesurée par le capteur pressBME et fournit la variable décrivant  la pression au niveau de la mer seaLevelPress. On voit que les variables _locales_ utilisées par la fonction (m_Pressure, sl_Pressure) n'ont pas forcément le même nom que les variables _globales_ du programme principal (pressBME, seaLevelPress), cela permet de manipuler des noms de variables signifiants et cohérents avec leur contexte. Vous noterez que pouur souligner cette différence le nom des variables _locales à la fonction_ utilisent un système de séparation des mots (des "\_") différent de celui du programme principal ( le CamelCase).

In [7]:
# déclarations
import mraa, time, math
from upm import pyupm_bmp280 as bmp
from upm import pyupm_jhd1313m1 as jdm

lcdAddress = 0x3E
rgbAddress = 0x62
bus1 = 0
bmeAddr = 0x77

bme =bmp.BME280(bus1,bmeAddr)
lcd = jdm.Jhd1313m1(bus1,lcdAddress,rgbAddress)
status = lcd.clear()
status = lcd.backlightOff()

In [8]:
# définition de l'altitude locale
localAlt = 156.5
# Calcul de la pression au niveau de la mer
def convertSeaLevel (m_Pressure):
    sl_Pressure = m_Pressure / math.pow(1.0 - localAlt/44330, 5.255)
    return sl_Pressure
# fin du calcul

In [9]:
# dessin et enregistrement du caractère degré
degree = (
    0b01100,
    0b10010,
    0b01100,
    0b00000,
    0b00000,
    0b00000,
    0b00000,
    0b00000)

lcd.createChar(0,degree)

0

In [ ]:
# Mise à jour du capteur
bme.update()
#Triple mesure
tempBME = bme.getTemperature()
pressBME = bme.getPressure() / 100.0
hygroBME = bme.getHumidity()
# calcul de la pression au niveau de la mer
seaLevelPress = convertSeaLevel (pressBME)
#impression des valeurs mesurées dans le cahier
print ('Humidité : \tTempérature : \tPression : \tPression(mer) : ')
print ("{0:.2f} %\t\t{1:.2f} °C\t{2:.2f} hPa\t{3:.2f} hPa".format(hygroBME,tempBME,pressBME,seaLevelPress))
# affichage des valeurs mesurées sur l'afficheur
status = lcd.backlightOff()
status = lcd.clear()
lcd.setColor(128,0,192)
lcd.setCursor(1, 0)
status = lcd.backlightOn()
status = lcd.write("Sea   : {0:6.2f} hPa".format(seaLevelPress))
status = lcd.write(" Hum  : {0:.2f} %".format(hygroBME))
lcd.setCursor(2, 0)
status = lcd.write("Press :  {0:6.2f} hPa".format(pressBME))
status = lcd.write(" Temp : {0:.2f} ".format(tempBME))
status = lcd.write(chr(0))
status = lcd.write("C")
for i in range(0,68,1) :
    lcd.scroll(True)
    time.sleep(0.5)
time.sleep(1)
status = lcd.clear()
status = lcd.backlightOff()

Humidité : 	Température : 	Pression : 	Pression(mer) : 
42.35 %		24.18 °C	985.31 hPa	1003.79 hPa
